<a href="https://colab.research.google.com/github/anindabitm/Zindi_hack/blob/master/Zindi_hack3_nb_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [2]:
#Mount google drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Zindi_Hack/3/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# import statements
import os
import sys
import csv
import shutil
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import random
%matplotlib inline

In [0]:
#Install fastai2
!pip install fastai2

In [0]:
#import fastai modules
from fastai2.torch_basics import *
from fastai2.basics import *
from fastai2.data.all import *
from fastai2.callback.all import *
from fastai2.vision.all import *
from fastai2.test_utils import *
from fastai2.vision.core import *

In [0]:
#seed everything
seed = 42
set_seed(42)
torch.backends.cudnn.benchmark = False
# python RNG
random.seed(seed)
# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
# numpy RNG
np.random.seed(seed)

In [0]:
!ls

images			  submission_file_tta.csv
images.zip		  submission_file_tta_cv.csv
sample_sub_v2.csv	  submission_file_tta_cv_dense169.csv
submission_file.csv	  submission_file_tta_cv_mixup.csv
submission_file_tta1.csv  train_labels.csv


In [0]:
!unzip images.zip

In [7]:
import glob
fn=glob.glob('./images/*.*')
len(fn)

1817

In [6]:
train=pd.read_csv('./train_labels.csv')
train.shape

(1308, 2)

In [7]:
train.head()

,image,target
0,tfymlmdkpzkqdjhdxyhnoeuqszxphw.jpg,0
1,rxgismgsvmaayzjarbfjaljhqmpbrt.jpg,1
2,uuzshfrhkgrkolhwdvliqauzulurnz.jpg,0
3,mjspxsagzusaznvnyxgamtrlqkqklp.jpg,0
4,rlbmuajgezfiddjzlyeoupxpqubkpt.jpg,1


In [8]:
sub=pd.read_csv('./sample_sub_v2.csv')
print(sub.shape)
sub.head()

(509, 2)


,image,target
0,aadawlxbmapqrblgxyzarhjasgiobu.png,0
1,abpxvdfyhaaohzcrngcucmhffwizxs.jpg,0
2,aclkjfvackiieiznzfcwienplielrj.jpg,0
3,aelkivmayxgsdjosiibfgmkvfrjvjk.jpg,0
4,aelzzshpfxerelefnkatpczktuxjln.jpg,0


In [0]:
#Create dataloader 
dls = ImageDataLoaders.from_df(train, path='.', folder='images', seed=42, item_tfms=Resize(224),batch_tfms = [*aug_transforms(size=224), Normalize.from_stats(*imagenet_stats)],bs=8,num_workers=0) 

In [10]:
#Fairly balanced dataset
train['target'].value_counts()

0    664
1    644
Name: target, dtype: int64

In [33]:
#Instantiate and start learner tuning
learn = cnn_learner(dls,densenet201,metrics=[accuracy])
learn.fine_tune(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.796428,0.347297,0.904215,01:07


epoch,train_loss,valid_loss,accuracy,time
0,0.403690,0.216865,0.946360,01:31
1,0.339511,0.121378,0.957854,01:31
2,0.295669,0.089138,0.969349,01:31
3,0.188126,0.130703,0.950192,01:31
4,0.192058,0.096920,0.950192,01:31
5,0.162544,0.070878,0.973180,01:31
6,0.114209,0.051254,0.984674,01:31
7,0.115087,0.053931,0.988506,01:30
8,0.052816,0.077222,0.973180,01:30
9,0.083990,0.048801,0.988506,01:30


In [0]:
test=sub.copy()
test.drop(['target'],axis=1,inplace=True)
test_dl=learn.dls.test_dl(test)
test_dl.show_batch()

In [38]:
# Using TTA to make robust prediction
preds,_=learn.tta(dl=test_dl)

In [0]:
# Check shape of prediction
preds.numpy().shape

In [0]:
dls.vocab

In [35]:
sub_file=sub.copy()
sub_file['target']=preds.numpy()[:,1]
sub_file.shape

(509, 2)

In [0]:
#Write to file
sub_file.to_csv('./submission_file_tta1.csv',index=False)